# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [28]:
import os
import numpy as np
import joblib
from azureml.core import Workspace, Experiment
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model

## Dataset

### Overview

In this project, we are going to classify the progressive muscle weakness and disease with the use of SKLearn Classifier and AutoML.

The dataset is custom data curated from one of the health institution.

The dataset contains 3 categories:
1. Control
    
    Indicates whether muscle weakness is in control or not.
    
    
2. SPG4
    
    **S pastic paraplegia 4 (SPG4)** is the most common type of hereditary spastic paraplegia (HSP) inherited in an autosomal dominant manner. Disease onset ranges from infancy to older adulthood. SPG4 is characterized by slowly progressive muscle weakness and spasticity (stiff or rigid muscles) in the lower half of the body. In rare cases, individuals may have a more complex form with seizures, ataxia, and dementia. SPG4 is caused by mutations in the SPAST gene. Severity of symptoms usually worsens over time, however some individuals remain mildly affected throughout their lives. Medications, such as antispastic drugs and physical therapy may aid in stretching spastic muscles and preventing contractures (fixed tightening of muscles) 
    
    Source : https://rarediseases.info.nih.gov/diseases/4925/spastic-paraplegia-4
    
    
3. Disease
   
   Indicates that the muscle weakness is there due to some desease.


In [4]:
path = "https://0547078f50ce.ngrok.io/data.csv"

ds = TabularDatasetFactory.from_delimited_files(path,
                                                validate=True,
                                                include_path=False,
                                                infer_column_types=True,
                                                separator=',',
                                                header=True,
                                                support_multi_line=False,
                                                empty_as_string=False)


In [9]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mwd-automl'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-138262
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-138262


In [10]:

try:
    cpu_cluster = ComputeTarget(workspace=ws, name="mwd-compute")
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,"mwd-compute", compute_config)

Found existing cluster, use it.


## AutoML Configuration


1. experiment_timeout_minutes : To stop the experiment by giving duration in minutes.
2. primary_metric : This is the the target on which our Auto ML config tries to improve.
3. enable_early_stopping: We have used this config to stop the experiment, if the primary_metric not improving.
4. enable_voting_ensemble: We don't want to use VotingClassifier as it a combined result of multiple classifier.
5. enable_stack_ensemble: We also disabled this option as it is stacked resilt of multiple classifier.

In [11]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "primary_metric": 'accuracy',
    "n_cross_validations": 5,
    "max_concurrent_iterations": 4,
    "enable_early_stopping": True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task="classification",
                            compute_target=cpu_cluster,
                            training_data=ds,
                            label_column_name="Output",
                            **automl_settings)


In [12]:
auto_ml_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In [18]:
RunDetails(auto_ml_run).show()
auto_ml_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|1982                             |SPG4                             |77607                                 |
+---------------------------------+----------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION 

{'runId': 'AutoML_28268a08-9643-4d77-84d6-ff7f89e0ec72',
 'target': 'mwd-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-09T14:33:30.633083Z',
 'endTimeUtc': '2021-02-09T15:21:56.680881Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'mwd-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"mwd-automl","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-138262","workspace_name":"quick-starts-ws-138262","region":"southcentralus","compute_target":"mwd-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"feat

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [19]:
# Retrieve and save your best automl model.

best_run, best_model = auto_ml_run.get_output()

best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


norm_macro_recall 0.69610639627683
accuracy 0.9788162943877733
precision_score_weighted 0.975797015097897
recall_score_micro 0.9788162943877733
precision_score_micro 0.9788162943877733
f1_score_micro 0.9788162943877733
recall_score_macro 0.7974042641845533
AUC_weighted 0.9992258632855819
f1_score_weighted 0.9765576653431175
recall_score_weighted 0.9788162943877733
AUC_macro 0.9958976149031014
average_precision_score_micro 0.998922409162803
precision_score_macro 0.86631292898749
average_precision_score_macro 0.8716601930172736
weighted_accuracy 0.9939731970116057
log_loss 0.046501558149861504
AUC_micro 0.9994530173547848
f1_score_macro 0.8229774817998374
matthews_correlation 0.9588881058847024
balanced_accuracy 0.7974042641845533
average_precision_score_weighted 0.9895669636024567
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_28268a08-9643-4d77-84d6-ff7f89e0ec72_28/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_28268a08-9643-4d77-84d6-ff7f89e0ec72

In [20]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('StandardScalerWrapper',
  <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper at 0x7f253a2dae10>),
 ('XGBoostClassifier',
  XGBoostClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                    colsample_bynode=1, colsample_bytree=0.9, eta=0.001, gamma=0,
                    grow_policy='lossguide', learning_rate=0.1, max_bin=255,
                    max_delta_step=0, max_depth=10, max_leaves=7,
                    min_child_weight=1, missing=nan, n_estimators=100, n_jobs=1,
                    nthread=None, objective='multi:softprob', random_state=0,
                    reg

In [21]:
best_run.download_file("/outputs/model.pkl", "automl_job_classification-best-model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
model = best_run.register_model(experiment_name+"-model", model_path="outputs/model.pkl")


In [24]:
best_run.download_file('outputs/model.pkl', './model.pkl')

In [26]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', './score.py')

In [29]:
inference_config = InferenceConfig(entry_script='./score.py')

service = Model.deploy(ws, experiment_name+"-service", [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [33]:
!python endpoint.py

b'"{\\"result\\": [\\"Disease\\"]}"'


TODO: In the cell below, print the logs of the web service and delete the service

In [34]:
service.update(enable_app_insights=True)
service.get_logs()

'2021-02-09T15:41:55,742627800+00:00 - iot-server/run \n2021-02-09T15:41:55,839630500+00:00 - rsyslog/run \n2021-02-09T15:41:55,852329000+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-02-09T15:41:55,959736200+00:00 - gunicorn/run 

In [40]:
service.delete()